In [1]:
!nvidia-smi

Wed May  5 07:10:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks

/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks


This notebook shows how to fine-tune a BERT model (from huggingface) for our dataset recognition task.

Note that internet is needed during the training phase (for downloading the bert-base-cased model). Internet can be turned off during prediction.

## Install packages

In [4]:
#!pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
!pip install datasets --no-index --find-links=file:///content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets

Looking in links: file:///content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/datasets-1.5.0-py3-none-any.whl
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/xxhash-2.0.0-cp37-cp37m-manylinux2010_x86_64.whl
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/huggingface_hub-0.0.7-py3-none-any.whl
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/fsspec-0.8.7-py3-none-any.whl


In [5]:
!pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
!pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

Processing /content/drive/My Drive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
Processing /content/drive/My Drive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
Processing /content/drive/My Drive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl
     |████████████████████████████████| 901kB 11.9MB/s 


# Import

In [6]:
import os
import re
import json
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import gc

random.seed(123)
np.random.seed(456)

In [7]:
# copy my_seqeval.py to the working directory because the input directory is non-writable
#!cp /kaggle/input/coleridge-packages/my_seqeval.py ./
!cp ../input/coleridge-packages/my_seqeval.py ./

In [8]:
!ls

eda-coleridge-initiative.ipynb
kagglenb003-annotation-data.ipynb
kagglenb005-pytorch-bert-for-ner.ipynb
l2knb001-transformers-ner
localnb001-transformers-ner
localnb001-transformers-ner.ipynb
my_seqeval.py
my_seqeval.py.lock
nb003-annotation-data
nb003-annotation-data.ipynb
nb005-pytorch-bert-for-ner-512
nb005-pytorch-bert-for-ner.ipynb
NERDA
output_folder
pytorch-bert-for-named-entity-recognition.ipynb
pytorch-xla-env-setup.py
setup_kaggle.ipynb
torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl
train_ner.json


# Hyper-parameters

In [9]:
MAX_LENGTH = 512 #64 # max no. words for each sentence.
OVERLAP = 170 #20 # if a sentence exceeds MAX_LENGTH, we split it to multiple sentences with overlapping

MAX_SAMPLE = None # set a small number for experimentation, set None for production.

# Load data

In [10]:
train_path = '../input/coleridgeinitiative-show-us-the-data/train.csv'
paper_train_folder = '../input/coleridgeinitiative-show-us-the-data/train'

train = pd.read_csv(train_path)
train = train[:MAX_SAMPLE]
print(f'No. raw training rows: {len(train)}')

No. raw training rows: 19661


Group by publication, training labels should have the same form as expected output.

In [11]:
train = train.groupby('Id').agg({
    'pub_title': 'first',
    'dataset_title': '|'.join,
    'dataset_label': '|'.join,
    'cleaned_label': '|'.join
}).reset_index()

print(f'No. grouped training rows: {len(train)}')

No. grouped training rows: 14316


In [12]:
#papers = {}
#for paper_id in train['Id'].unique():
#    with open(f'{paper_train_folder}/{paper_id}.json', 'r') as f:
#        paper = json.load(f)
#        papers[paper_id] = paper

#!cp -r {paper_train_folder} /content
tmp = pd.read_pickle("../input/kagglenb007-get-text/df_train.pkl")
tmp.drop_duplicates(subset=["Id"], keep='first', ignore_index=True, inplace=True)
papers = {k:v for k,v in zip(tmp["Id"], tmp["text"])}

del tmp
gc.collect()

0

# Transform data to NER format

In [13]:
def clean_training_text(txt):
    """
    similar to the default clean_text function but without lowercasing.
    """
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt)).strip()

def shorten_sentences(sentences):
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

def find_sublist(big_list, small_list):
    all_positions = []
    for i in range(len(big_list) - len(small_list) + 1):
        if small_list == big_list[i:i+len(small_list)]:
            all_positions.append(i)
    
    return all_positions

def tag_sentence(sentence, labels): # requirement: both sentence and labels are already cleaned
    sentence_words = sentence.split()
    
    if labels is not None and any(re.findall(f'\\b{label}\\b', sentence)
                                  for label in labels): # positive sample
        nes = ['O'] * len(sentence_words)
        for label in labels:
            label_words = label.split()

            all_pos = find_sublist(sentence_words, label_words)
            for pos in all_pos:
                nes[pos] = 'B'
                for i in range(pos+1, pos+len(label_words)):
                    nes[i] = 'I'

        return True, list(zip(sentence_words, nes))
        
    else: # negative sample
        nes = ['O'] * len(sentence_words)
        return False, list(zip(sentence_words, nes))

In [14]:
papers['d0fa7568-7d8e-4db9-870f-f9c6f668c17b']

[{'section_title': 'What is this study about?',
  'text': 'This study used data from the National Education Longitudinal Study (NELS:88) to examine the effects of dual enrollment programs for high school students on college degree attainment. The study also reported whether the impacts of dual enrollment programs were different for first generation college students versus students whose parents had attended at least some college. In addition, a supplemental analysis reports on the impact of different amounts of dual enrollment course-taking and college degree attainment.\nDual enrollment programs offer college-level learning experiences for high school students. The programs offer college courses and/or the opportunity to earn college credits for students while still in high school.\nThe intervention group in the study was comprised of NELS participants who attended a postsecondary school and who participated in a dual enrollment program while in high school (n = 880). The study author

In [15]:
cnt_pos, cnt_neg = 0, 0 # number of sentences that contain/not contain labels
ner_data = []

pbar = tqdm(total=len(train))
for i, id, dataset_label in train[['Id', 'dataset_label']].itertuples():
    # paper
    paper = papers[id]
    
    # labels
    labels = dataset_label.split('|')
    labels = [clean_training_text(label) for label in labels]
    
    # sentences
    sentences = set([clean_training_text(sentence) for section in paper 
                 for sentence in section['text'].split('.') 
                ])
    sentences = shorten_sentences(sentences) # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    
    # positive sample
    for sentence in sentences:
        is_positive, tags = tag_sentence(sentence, labels)
        if is_positive:
            cnt_pos += 1
            ner_data.append(tags)
        elif any(word in sentence.lower() for word in ['data', 'study']): 
            ner_data.append(tags)
            cnt_neg += 1
    
    # process bar
    pbar.update(1)
    pbar.set_description(f"Training data size: {cnt_pos} positives + {cnt_neg} negatives")

# shuffling
random.shuffle(ner_data)

Training data size: 47202 positives + 514263 negatives: 100%|██████████| 14316/14316 [02:43<00:00, 95.66it/s]

write data to file.

In [16]:
with open('train_ner.json', 'w') as f:
    for row in ner_data:
        words, nes = list(zip(*row))
        row_json = {'tokens' : words, 'tags' : nes}
        json.dump(row_json, f)
        f.write('\n')

Training data size: 47202 positives + 514263 negatives: 100%|██████████| 14316/14316 [03:00<00:00, 95.66it/s]

# Fine-tune a BERT model for NER

In [17]:
output_folder = "nb005-pytorch-bert-for-ner"
!rm -r {output_folder}
!mkdir {output_folder}
os.environ["OUTPUT_DIR"] = output_folder

def train(resume_training=False, num_checkpoint=None):
    if resume_training:
        os.environ["MODEL_PATH"] = f"./{output_folder}/checkpoint-{num_checkpoint}"
        !python ../input/kaggle-ner-utils/kaggle_run_ner.py \
        --model_name_or_path $MODEL_PATH \
        --train_file './train_ner.json' \
        --validation_file './train_ner.json' \
        --num_train_epochs 5 \
        --per_device_train_batch_size 8 \
        --per_device_eval_batch_size 8 \
        --save_steps 15000 \
        --output_dir $OUTPUT_DIR \
        --report_to 'none' \
        --seed 123 \
        --do_train 
    else:
        !python ../input/kaggle-ner-utils/kaggle_run_ner.py \
        --model_name_or_path 'bert-base-cased' \
        --train_file './train_ner.json' \
        --validation_file './train_ner.json' \
        --num_train_epochs 5 \
        --per_device_train_batch_size 8 \
        --per_device_eval_batch_size 8 \
        --save_steps 15000 \
        --output_dir $OUTPUT_DIR \
        --report_to 'none' \
        --seed 123 \
        --do_train 

In [18]:
 train(resume_training=False)
#train(resume_training=True, num_checkpoint=45000)

2021-05-04 10:21:03.639834: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/04/2021 10:21:05 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
05/04/2021 10:21:05 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=nb005-pytorch-bert-for-ner, overwrite_output_dir=False, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/May04_10-21-05_1696736c7213, logging_strategy=IntervalStrategy.STEPS, logging_f

After the tuning finishes, we should find our model in './nb005-pytorch-bert-for-ner'.

# Upload to Kaggle
./nb005-pytorch-bert-for-nerをKaggle datasetsにupload

In [19]:
!pip install --upgrade --force-reinstall --no-deps kaggle
f = open("/content/drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
json_data = json.load(f)
os.environ["KAGGLE_USERNAME"] = json_data["username"]
os.environ["KAGGLE_KEY"] = json_data["key"]

     |████████████████████████████████| 61kB 5.4MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=2312eee6e13e071adec9db2a2ec84a845573093985ab2b6e90ed9195389cecd0
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [23]:
!ls

eda-coleridge-initiative.ipynb
kagglenb003-annotation-data.ipynb
kagglenb005-pytorch-bert-for-ner.ipynb
l2knb001-transformers-ner
localnb001-transformers-ner
localnb001-transformers-ner.ipynb
my_seqeval.py
my_seqeval.py.lock
nb003-annotation-data
nb003-annotation-data.ipynb
nb005-pytorch-bert-for-ner-512
nb005-pytorch-bert-for-ner.ipynb
NERDA
output_folder
pytorch-bert-for-named-entity-recognition.ipynb
pytorch-xla-env-setup.py
setup_kaggle.ipynb
torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl
train_ner.json


In [24]:
dname = "nb005-pytorch-bert-for-ner-512"
#!mkdir {dname}
!kaggle datasets init -p {dname}
# referene: https://kaeru-nantoka.hatenablog.com/entry/2020/01/17/015551

with open(f"{dname}/dataset-metadata.json", "r") as jsonFile:
    data = json.load(jsonFile)

data["id"] = f"riow1983/{dname}"
data["title"] = dname

with open(f"{dname}/dataset-metadata.json", "w") as jsonFile:
    json.dump(data, jsonFile)

# Copy train_ner.json
!cp train_ner.json ./{dname}/train_ner.json
!kaggle datasets create -p {dname}
#!kaggle datasets version -p {dname} -m "[] hogehoge"

Data package template written to: nb005-pytorch-bert-for-ner-512/dataset-metadata.json
Skipping folder: checkpoint-15000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-30000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-45000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-60000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-75000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-90000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-105000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-120000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-135000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-150000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-165000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-180000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-195000; use '--dir-mode' to upload fold

In [ ]:
# Copy train_ner.json
#!cp train_ner.json ./{dname}/train_ner.json
#!kaggle datasets version -p {dname} -m "[Add] train_ner.json"

Skipping folder: checkpoint-15000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-30000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-45000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-60000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-75000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-90000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-105000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-120000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-135000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-150000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-165000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-180000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-195000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-210000; use '--dir-mode' to upload folders
Skipping fol